# 모듈 7: 멀티 에이전트(Multi-Agent) 솔루션 구축하기

이 모듈에서는 하나의 거대 에이전트 대신, 여러 전문 에이전트가 협력하여 복잡한 문제를 해결하는 **멀티 에이전트 시스템**을 구축하는 방법을 배운다.

## 학습 목표

- **전문 에이전트**: 특정 도메인(날씨, 여행 등)에 특화된 에이전트를 만든다.
- **Agent-as-a-Tool**: 에이전트를 다른 에이전트의 도구로 변환하는 `AgentTool` 패턴을 이해한다.
- **위임(Delegation)**: 루트 에이전트가 하위 에이전트에게 작업을 맡기는 워크플로우를 구현한다.

## 멀티 에이전트 시스템이란

복잡한 작업은 한 명의 천재보다 전문가 팀이 더 잘 해결할 수 있다. ADK에서는 다음과 같은 구조로 팀을 만든다.

1.  **루트 에이전트 (Manager)**: 사용자의 요청을 듣고, 어떤 전문가가 필요한지 판단하여 작업을 지시한다.
2.  **서브 에이전트 (Specialist)**: 지시받은 특정 작업만 전문적으로 수행하고 결과를 보고한다.

ADK에서는 **서브 에이전트를 도구(Tool)처럼 등록**함으로써, 루트 에이전트가 함수를 호출하듯 에이전트를 호출하게 만든다.

In [ ]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv

In [1]:
import os
from dotenv import load_dotenv

from google.adk import Agent
from google.adk.tools import AgentTool
from google.adk.runners import InMemoryRunner
import random

load_dotenv()
MODEL = "gemini-2.5-flash"

True

## 1단계: 전문 에이전트(Sub-Agents) 만들기

먼저 각자의 역할에 충실한 전문 에이전트 두 명을 만든다.
1.  **weather_agent**: 날씨 정보를 조회하는 에이전트
2.  **travel_agent**: 여행 일정과 팁을 제공하는 에이전트

In [2]:
# 1. 날씨 조회 함수 (도구)
def get_weather(location: str) -> dict:
    """특정 위치의 현재 날씨 정보를 가져옵니다."""
    print(f"[System] 날씨 데이터 조회 중: {location}...")
    conditions = random.choice(["맑음", "구름 많음", "비", "눈"])
    temp = round(random.uniform(10, 30), 1)
    return {"location": location, "temperature": f"{temp}°C", "condition": conditions}

# 2. 날씨 에이전트 정의
weather_agent = Agent(
    name="weather_agent",
    model=MODEL,
    description="날씨 정보를 제공하는 전문 에이전트",
    instruction="""
    당신은 기상 캐스터입니다.
    사용자가 특정 지역의 날씨를 물어보면 'get_weather' 도구를 사용하여 정보를 확인하고 알려주세요.
    """,
    tools=[get_weather]
)

# 3. 여행 에이전트 정의 (별도 도구 없이 지식 기반 답변)
travel_agent = Agent(
    name="travel_agent",
    model=MODEL,
    description="여행 일정 및 관광지 추천 전문 에이전트",
    instruction="""
    당신은 여행 가이드입니다.
    관광지, 맛집, 숙박 추천 등 여행과 관련된 유용한 정보를 제공하세요.
    항상 설레는 톤으로 대화하세요.
    """
)

## 2단계: 루트 에이전트(Root Agent) 만들기

이제 팀장 역할을 할 `vacation_planner`를 만든다. 핵심은 **`tools` 리스트에 `AgentTool(에이전트)`를 넣는 것**이다.

이렇게 하면 루트 에이전트는 필요할 때 `weather_agent`나 `travel_agent`를 마치 함수처럼 호출하여 작업을 위임한다.

In [3]:
vacation_planner = Agent(
    name="vacation_planner",
    model=MODEL,
    description="사용자의 휴가 계획을 총괄하는 매니저 에이전트",
    instruction="""
    당신은 휴가 계획을 도와주는 메인 매니저입니다.
    사용자의 요청을 분석하여 다음과 같이 전문 에이전트에게 작업을 위임(Delegate)하세요.
    
    [위임 규칙]
    1. 날씨 정보가 필요하면 -> 'weather_agent' 도구를 호출하세요.
    2. 관광지 추천이나 여행 팁이 필요하면 -> 'travel_agent' 도구를 호출하세요.
    3. 두 정보가 모두 필요하면, 각각 호출하여 정보를 수집한 뒤 종합하여 답변하세요.
    
    직접 답변을 지어내지 말고, 반드시 전문 에이전트의 답변을 참고하여 최종 응답을 작성하세요.
    """,
    # 중요: 서브 에이전트를 도구로 등록
    tools=[
        AgentTool(weather_agent), 
        AgentTool(travel_agent)
    ]
)

## 3단계: 시스템 실행 (Orchestration)

`InMemoryRunner`에는 루트 에이전트만 등록하면 된다. 하위 에이전트는 루트 에이전트의 도구로서 자동으로 관리된다.

In [4]:
# Runner 생성 (루트 에이전트만 전달)
runner = InMemoryRunner(agent=vacation_planner)

print("멀티 에이전트 시스템이 준비되었습니다.")

멀티 에이전트 시스템이 준비되었습니다.


## 협업 테스트

복합적인 질문을 던져서 루트 에이전트가 어떻게 하위 에이전트들을 호출하고 정보를 취합하는지 확인한다.

In [5]:
# 복합 요청: 날씨 정보(weather_agent)와 여행 정보(travel_agent)가 모두 필요한 질문
await runner.run_debug("파리로 휴가를 가려고 해. 그곳 날씨는 어떻고, 어디를 가면 좋을까?")


 ### Created new session: debug_session_id

User > 파리로 휴가를 가려고 해. 그곳 날씨는 어떻고, 어디를 가면 좋을까?


[System] 날씨 데이터 조회 중: 파리...
vacation_planner > 현재 파리에는 비가 내리고 있으며, 온도는 10.0°C입니다. 파리의 대표적인 랜드마크로는 에펠탑, 루브르 박물관, 개선문, 베르사유 궁전, 몽마르뜨 언덕이 있습니다. 숨겨진 장소로는 생 마르탱 운하, 마레 지구, 페르 라셰즈 묘지가 있습니다. 파리는 대중교통이 잘 되어 있고, 파리 뮤지엄 패스를 구입하면 여러 박물관과 미술관을 할인된 가격으로 입장할 수 있습니다. 소매치기를 조심하세요.



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'request': '파리 날씨'
         },
         id='adk-6465dc95-7005-4a59-a24c-43931b80bc04',
         name='weather_agent'
       )
     ),
     Part(
       function_call=FunctionCall(
         args={
           'request': '파리 관광지 추천'
         },
         id='adk-0238aaaf-138d-460b-b06a-75a18fcf5125',
         name='travel_agent'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=19,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=19
     ),
   ],
   prompt_token_count=295,
   prompt_tokens_details=[
     ModalityTokenCount(
       mo

## 동작 원리 분석

로그를 살펴보면 다음과 같은 흐름으로 처리가 진행된다.

1.  **User**: "파리 휴가... 날씨... 어디 가면..."
2.  **Root Agent**: 질문 분석 -> 날씨와 여행 정보가 필요함을 인지.
3.  **Root Agent**: `weather_agent` 도구 호출 -> 파리 날씨 정보 획득.
4.  **Root Agent**: `travel_agent` 도구 호출 -> 파리 관광지 추천 획득.
5.  **Root Agent**: 두 정보를 종합하여 최종 답변 생성.

## 정리

이 모듈에서는 **AgentTool**을 사용하여 에이전트 간의 위임(Delegation)을 구현했다. 이 방식은 복잡한 시스템을 모듈화하고 유지보수하기 쉽게 만든다.

다음 모듈에서는 시스템의 안정성을 높이기 위해, 에이전트 실행 과정을 가로채고 수정할 수 있는 **콜백(Callback)**과 **가드레일(Guardrails)**에 대해 배울 것이다.